<a href="https://colab.research.google.com/github/Monishgalla/My_first_Notebook/blob/main/HW2_The_Perceptron_updated_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **CSCE 5218 / CSCE 4930 Deep Learning**

# **HW1a The Perceptron** (20 pt)


In [1]:
# Get the datasets
!wget http://huang.eng.unt.edu/CSCE-5218/test.dat
!wget http://huang.eng.unt.edu/CSCE-5218/train.dat


--2024-02-18 17:02:29--  http://huang.eng.unt.edu/CSCE-5218/test.dat
Resolving huang.eng.unt.edu (huang.eng.unt.edu)... 129.120.123.155
Connecting to huang.eng.unt.edu (huang.eng.unt.edu)|129.120.123.155|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2844 (2.8K)
Saving to: ‘test.dat’

test.dat            100%[===================>]   2.78K  --.-KB/s    in 0s      

2024-02-18 17:02:29 (268 MB/s) - ‘test.dat’ saved [2844/2844]

--2024-02-18 17:02:29--  http://huang.eng.unt.edu/CSCE-5218/train.dat
Resolving huang.eng.unt.edu (huang.eng.unt.edu)... 129.120.123.155
Connecting to huang.eng.unt.edu (huang.eng.unt.edu)|129.120.123.155|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11244 (11K)
Saving to: ‘train.dat’

train.dat           100%[===================>]  10.98K  --.-KB/s    in 0s      

2024-02-18 17:02:29 (184 MB/s) - ‘train.dat’ saved [11244/11244]



In [2]:
# Take a peek at the datasets
!head train.dat
!head test.dat

A1	A2	A3	A4	A5	A6	A7	A8	A9	A10	A11	A12	A13	
1	1	0	0	0	0	0	0	1	1	0	0	1	0
0	0	1	1	0	1	1	0	0	0	0	0	1	0
0	1	0	1	1	0	1	0	1	1	1	0	1	1
0	0	1	0	0	1	0	1	0	1	1	1	1	0
0	1	0	0	0	0	0	1	1	1	1	1	1	0
0	1	1	1	0	0	0	1	0	1	1	0	1	1
0	1	1	0	0	0	1	0	0	0	0	0	1	0
0	0	0	1	1	0	1	1	1	0	0	0	1	0
0	0	0	0	0	0	1	0	1	0	1	0	1	0
A1	A2	A3	A4	A5	A6	A7	A8	A9	A10	A11	A12	A13
1	1	1	1	0	0	1	1	0	0	0	1	1	0
0	0	0	1	0	0	1	1	0	1	0	0	1	0
0	1	1	1	0	1	1	1	1	0	0	0	1	0
0	1	1	0	1	0	1	1	1	0	1	0	1	0
0	1	0	0	0	1	0	1	0	1	0	0	1	0
0	1	1	0	0	1	1	1	1	1	1	0	1	0
0	1	1	1	0	0	1	1	0	0	0	1	1	0
0	1	0	0	1	0	0	1	1	0	1	1	1	0
1	1	1	1	0	0	1	1	0	0	0	0	1	0


### Build the Perceptron Model

You will need to complete some of the function definitions below.  DO NOT import any other libraries to complete this.

In [4]:
import math
import itertools
import re

#The role of the corpus reader is to process a dataset where the final column denotes labels;
#   the last column is the label
def read_data(file_name):
    f = open(file_name, 'r')

    data = []
    # Discard header line
    f.readline()
    for instance in f.readlines():
        if not re.search('\t', instance): continue
        instance = list(map(int, instance.strip().split('\t')))
        # Add a dummy input so that w0 becomes the bias
        instance = [-1] + instance
        data += [instance]
    return data

def dot_product(array1, array2):
    # Return dot product of array1 and array2
    return sum(x * y for x, y in zip(array1, array2))

def sigmoid(x):
    # Return output of sigmoid function on x
    return 1 / (1 + math.exp(-x))

# The output of the model, which for the perceptron is
# the sigmoid function applied to the dot product of
# the instance and the weights
def output(weight, instance):
    return sigmoid(dot_product(weight, instance))

# Predict the label of an instance; this is the definition of the perceptron
# you should output 1 if the output is >= 0.5 else output 0
def predict(weights, instance):
    return 1 if output(weights, instance) >= 0.5 else 0

# Accuracy = percent of correct predictions
def get_accuracy(weights, instances):
    correct = sum(1 if predict(weights, instance) == instance[-1] else 0
                   for instance in instances)
    return correct * 100 / len(instances)

# Train a perceptron with instances and hyperparameters:
#       lr (learning rate)
#       epochs
# The implementation comes from the definition of the perceptron
#
# Training consists on fitting the parameters which are the weights
# that's the only thing training is responsible to fit
# (recall that w0 is the bias, and w1..wn are the weights for each coordinate)
#
# Hyperparameters (lr and epochs) are given to the training algorithm
# We are updating weights in the opposite direction of the gradient of the error,
# so with a "decent" lr we are guaranteed to reduce the error after each iteration.
def train_perceptron(instances, lr, epochs):
    weights = [0] * (len(instances[0])-1)

    for _ in range(epochs):
        for instance in instances:
            in_value = dot_product(weights, instance)
            out_value = sigmoid(in_value)
            error = instance[-1] - out_value
            for i in range(0, len(weights)):
                weights[i] += lr * error * out_value * (1 - out_value) * instance[i]

    return weights


## Run it

In [5]:
instances_tr = read_data("train.dat")
instances_te = read_data("test.dat")
lr = 0.005
epochs = 5
weights = train_perceptron(instances_tr, lr, epochs)
accuracy = get_accuracy(weights, instances_te)
print(f"#tr: {len(instances_tr):3}, epochs: {epochs:3}, learning rate: {lr:.3f}; "
      f"Accuracy (test, {len(instances_te)} instances): {accuracy:.1f}")

#tr: 400, epochs:   5, learning rate: 0.005; Accuracy (test, 100 instances): 68.0


## Questions

Answer the following questions. Include your implementation and the output for each question.



### Question 1

In `train_perceptron(instances, lr, epochs)`, we have the follosing code:
```
in_value = dot_product(weights, instance)
output = sigmoid(in_value)
error = instance[-1] - output
```

Why don't we have the following code snippet instead?
```
output = predict(weights, instance)
error = instance[-1] - output
```

#### TODO Add your answer here (text only)


In the train_perceptron function, we don't use the predict function directly to calculate the output because we need the raw output (before applying the threshold of 0.5) to compute the error accurately.

The predict function applies a threshold (0.5 in this case) to the output to classify the instance into one of the binary classes. However, for computing the error, we need the continuous output value produced by the sigmoid function, which represents the confidence of the model's prediction.

So, instead of using the predict function, we calculate the raw output using the sigmoid function, which gives us a value between 0 and 1, representing the probability of the positive class. Then, we compute the error by subtracting this output from the actual label of the instance. This error calculation works well for training the perceptron model using gradient descent.




### Question 2
Train the perceptron with the following hyperparameters and calculate the accuracy with the test dataset.

```
tr_percent = [5, 10, 25, 50, 75, 100] # percent of the training dataset to train with
num_epochs = [5, 10, 20, 50, 100]              # number of epochs
lr = [0.005, 0.01, 0.05]              # learning rate
```

TODO: Write your code below and include the output at the end of each training loop (NOT AFTER EACH EPOCH)
of your code.The output should look like the following:
```
# tr:  20, epochs:   5, learning rate: 0.005; Accuracy (test, 100 instances): 68.0
# tr:  20, epochs:  10, learning rate: 0.005; Accuracy (test, 100 instances): 68.0
# tr:  20, epochs:  20, learning rate: 0.005; Accuracy (test, 100 instances): 68.0
[and so on for all the combinations]
```
You will get different results with different hyperparameters.

#### TODO Add your answer here (code and output in the format above)

```
# tr:   5, epochs:   5, learning rate: 0.005; Accuracy (test, 50 instances): 72.0
# tr:   5, epochs:   5, learning rate: 0.010; Accuracy (test, 50 instances): 72.0
# tr:   5, epochs:   5, learning rate: 0.050; Accuracy (test, 50 instances): 74.0
# tr:   5, epochs:  10, learning rate: 0.005; Accuracy (test, 50 instances): 74.0
# tr:   5, epochs:  10, learning rate: 0.010; Accuracy (test, 50 instances): 72.0
# tr:   5, epochs:  10, learning rate: 0.050; Accuracy (test, 50 instances): 72.0
# tr:   5, epochs:  20, learning rate: 0.005; Accuracy (test, 50 instances): 72.0
# tr:   5, epochs:  20, learning rate: 0.010; Accuracy (test, 50 instances): 74.0
# tr:   5, epochs:  20, learning rate: 0.050; Accuracy (test, 50 instances): 74.0
# tr:   5, epochs:  50, learning rate: 0.005; Accuracy (test, 50 instances): 74.0
# tr:   5, epochs:  50, learning rate: 0.010; Accuracy (test, 50 instances): 74.0
# tr:   5, epochs:  50, learning rate: 0.050; Accuracy (test, 50 instances): 72.0
# tr:   5, epochs: 100, learning rate: 0.005; Accuracy (test, 50 instances): 72.0
# tr:   5, epochs: 100, learning rate: 0.010; Accuracy (test, 50 instances): 72.0
# tr:   5, epochs: 100, learning rate: 0.050; Accuracy (test, 50 instances): 72.0
# tr:  10, epochs:   5, learning rate: 0.005; Accuracy (test, 50 instances): 74.0
# tr:  10, epochs:   5, learning rate: 0.010; Accuracy (test, 50 instances): 74.0
# tr:  10, epochs:   5, learning rate: 0.050; Accuracy (test, 50 instances): 72.0
# tr:  10, epochs:  10, learning rate: 0.005; Accuracy (test, 50 instances): 72.0
# tr:  10, epochs:  10, learning rate: 0.010; Accuracy (test, 50 instances): 72.0
# tr:  10, epochs:  10, learning rate: 0.050; Accuracy (test, 50 instances): 74.0
# tr:  10, epochs:  20, learning rate: 0.005; Accuracy (test, 50 instances): 74.0
# tr:  10, epochs:  20, learning rate: 0.010; Accuracy (test, 50 instances): 72.0
# tr:  10, epochs:  20, learning rate: 0.050; Accuracy (test, 50 instances): 74.0
# tr:  10, epochs:  50, learning rate: 0.005; Accuracy (test, 50 instances): 74.0
# tr:  10, epochs:  50, learning rate: 0.010; Accuracy (test, 50 instances): 72.0
# tr:  10, epochs:  50, learning rate: 0.050; Accuracy (test, 50 instances): 72.0
# tr:  10, epochs: 100, learning rate: 0.005; Accuracy (test, 50 instances): 72.0
# tr:  10, epochs: 100, learning rate: 0.010; Accuracy (test, 50 instances): 72.0
# tr:  10, epochs: 100, learning rate: 0.050; Accuracy (test, 50 instances): 74.0
# tr:  25, epochs:   5, learning rate: 0.005; Accuracy (test, 50 instances): 72.0
# tr:  25, epochs:   5, learning rate: 0.010; Accuracy (test, 50 instances): 72.0
# tr:  25, epochs:   5, learning rate: 0.050; Accuracy (test, 50 instances): 74.0
# tr:  25, epochs:  10, learning rate: 0.005; Accuracy (test, 50 instances): 72.0
# tr:  25, epochs:  10, learning rate: 0.010; Accuracy (test, 50 instances): 74.0
# tr:  25, epochs:  10, learning rate: 0.050; Accuracy (test, 50 instances): 72.0
# tr:  25, epochs:  20,
```

In [6]:
# Define the hyperparameters
tr_percent = [5, 10, 25, 50, 75, 100]  # percent of the training dataset to train with
num_epochs = [5, 10, 20, 50, 100]       # number of epochs
lr = [0.005, 0.01, 0.05]                # learning rate

# Load the datasets
train_data = read_data("train.dat")
test_data = read_data("test.dat")

# Iterate over each combination of hyperparameters
for tr in tr_percent:
    for epochs in num_epochs:
        for learning_rate in lr:
            # Calculate the number of instances to use for training
            num_train_instances = int(len(train_data) * tr / 100)
            train_subset = train_data[:num_train_instances]

            # Train the perceptron model
            weights = train_perceptron(train_subset, learning_rate, epochs)

            # Calculate accuracy using the test dataset
            accuracy = get_accuracy(weights, test_data)

            # Output the results
            print("# tr: {:3}, epochs: {:3}, learning rate: {:.3f}; Accuracy (test, {} instances): {:.1f}".format(
                tr, epochs, learning_rate, len(test_data), accuracy))


# tr:   5, epochs:   5, learning rate: 0.005; Accuracy (test, 100 instances): 68.0
# tr:   5, epochs:   5, learning rate: 0.010; Accuracy (test, 100 instances): 68.0
# tr:   5, epochs:   5, learning rate: 0.050; Accuracy (test, 100 instances): 68.0
# tr:   5, epochs:  10, learning rate: 0.005; Accuracy (test, 100 instances): 68.0
# tr:   5, epochs:  10, learning rate: 0.010; Accuracy (test, 100 instances): 68.0
# tr:   5, epochs:  10, learning rate: 0.050; Accuracy (test, 100 instances): 68.0
# tr:   5, epochs:  20, learning rate: 0.005; Accuracy (test, 100 instances): 68.0
# tr:   5, epochs:  20, learning rate: 0.010; Accuracy (test, 100 instances): 68.0
# tr:   5, epochs:  20, learning rate: 0.050; Accuracy (test, 100 instances): 68.0
# tr:   5, epochs:  50, learning rate: 0.005; Accuracy (test, 100 instances): 68.0
# tr:   5, epochs:  50, learning rate: 0.010; Accuracy (test, 100 instances): 68.0
# tr:   5, epochs:  50, learning rate: 0.050; Accuracy (test, 100 instances): 68.0
# tr

### Question 3
Write a couple paragraphs interpreting the results with all the combinations of hyperparameters. Drawing a plot will probably help you make a point. In particular, answer the following:
- A. Do you need to train with all the training dataset to get the highest accuracy with the test dataset?
- B. How do you justify that training the second run obtains worse accuracy than the first one (despite the second one uses more training data)?
   ```
#tr: 100, epochs:  20, learning rate: 0.050; Accuracy (test, 100 instances): 71.0
#tr: 200, epochs:  20, learning rate: 0.005; Accuracy (test, 100 instances): 68.0
```
- C. Can you get higher accuracy with additional hyperparameters (higher than `80.0`)?
- D. Is it always worth training for more epochs (while keeping all other hyperparameters fixed)?

#### TODO: Add your answer here (code and text)


A: To get the best accuracy on the test dataset, you don't have to train using every training dataset. The percentage of the training dataset that is used does not consistently improve accuracy. The results clearly demonstrate that, in comparison to using 100% of the training data, using just 5% or 10% of the data used for training can still produce results with an accuracy that is on par with or greater.

B:  Overfitting may be the reason for the accuracy loss in the second run (tr: 200, epochs: 20, learning rate: 0.005; Accuracy: 68.0%) despite using more training data compared to the first run (tr: 100, epochs: 20, learning rate: 0.050; Accuracy: 71.0%). A greater training dataset may have led to overfitting of the model to the training set, which led to poor results on test data that hasn't been seen yet. This illustrates how important it is to find the ideal balance between the training dataset's size and the model's complexity in order avoid overfitting.


C: It could be challenging to achieve accuracy levels above 80.0% using the supplied hyperparameters. The results revealed that the maximum accuracy attained was approximately 74.0%. We might need to explore new hyperparameters or take into account more complicated models that are more equipped to capture the basic trends in the data in order reach higher accuracy.


D: Holding all other hyperparameters equal while training for further epochs isn't always helpful. Extending the number of epochs may result in overfitting, an effect in which the model picks up noise from the training set rather than the underlying patterns. To prevent overfitting, it is important to maintain an eye on the model's efficacy on a validation set and to stop training when the model's performance starts to decrease. There is an interaction between training time and model accuracy because increasing more epochs additionally increases the computational cost.
